<h2>Capstone Project - The Battle of the Neighborhoods</h2>
<h3>Applied Data Science Capstone by IBM/Coursera</h3>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

<p>The main aim of this project is to ease peoples choice making of a neighbhorhood in North York,Toronto.
For ease of accessing to Cafe, School, Colleges, Grocery Shops,Theatre, Hospital etc.
This Project aim to create an analysis of features for a people migrating to North York to search a best neighborhood as a comparative analysis between neighborhoods. 
It will help people to get awareness of the area and neighborhood before moving to a city.</p>

<h1>Data Description:</h1>
    <p>Data Link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</p>
    <p>Web Scraping this data will get Postal Code	,Borough,Neighbourhood of North York,Toronto</p>


<h5>Importing Libraries</h5>

In [53]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import geocoder
import folium
import requests
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

<h5>Data Extraction and Cleaning</h5>
<p>Using BeautifulSoup Scraping List of Postal Codes of Given Wikipedia Page. Link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

</p>

In [6]:
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)


200


<h5>Web Scraping  using the BeautifulSoup package.</h5>

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')
table=soup.find('table',{'class':"wikitable"})
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

<h5>Tranform the data into a _pandas_ dataframe</h5>


In [9]:
df=pd.read_html(str(table))
df=pd.DataFrame(df[0])
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


<h5>Cleaning DataFrame</h5>

In [10]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [14]:
df = df.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [15]:
df.describe()


,Postal Code,Borough,Neighbourhood
count,103,103,103
unique,103,10,99
top,M3A,North York,Downsview
freq,1,24,4


In [19]:
df = df.dropna()
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, York'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.69208000000003, -79.47862999999995]

In [23]:
postal_codes = df['Postal Code']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [24]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [31]:
df=df.rename(columns={"Postal Code": "Postal_Code"})
df

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.692080,-79.478630
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.692080,-79.478630
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.692080,-79.478630
3,M1G,Scarborough,Woburn,43.692080,-79.478630
4,M1H,Scarborough,Cedarbrae,43.692080,-79.478630
...,...,...,...,...,...
98,M9N,York,Weston,43.705070,-79.518040
99,M9P,Etobicoke,Westmount,43.700717,-79.553168
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.700717,-79.553168
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.744530,-79.586240


In [42]:
NY_data = df[df['Borough'].str.contains("North York")].reset_index(drop=True)
NY_data.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.80225,-79.355580
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.69208,-79.478630
2,M2K,North York,Bayview Village,43.69208,-79.478630
3,M2L,North York,"York Mills, Silver Hills",43.74532,-79.387317
4,M2M,North York,"Willowdale, Newtonbrook",43.79182,-79.413720


In [96]:
address = 'North York,Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York,Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York,Toronto are 43.7543263, -79.44911696639593.


<h3>Map of North York</h3>

In [44]:
NY = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(NY_data['Latitude'], NY_data['Longitude'], NY_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NY)  
NY

In [45]:
CLIENT_ID = 'JHEGIUCWXRKGHICIK4KQHDR2PAOBRMFRZ42A3YMDKYEDBGV4' 
CLIENT_SECRET = 'QBLN55MW0KA5WSAMHGUSRMDCNGWBGT2I3XL3ANP0BP4NUIG0'
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JHEGIUCWXRKGHICIK4KQHDR2PAOBRMFRZ42A3YMDKYEDBGV4
CLIENT_SECRET:QBLN55MW0KA5WSAMHGUSRMDCNGWBGT2I3XL3ANP0BP4NUIG0


In [46]:
NY_data.loc[0, 'Neighbourhood']

'Hillcrest Village'

In [48]:
neighbourhood_latitude = NY_data.loc[0, 'Latitude'] 
neighbourhood_longitude = NY_data.loc[0, 'Longitude'] 

neighbourhood_name = NY_data.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.80225000000007, -79.35557999999997.


In [49]:
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JHEGIUCWXRKGHICIK4KQHDR2PAOBRMFRZ42A3YMDKYEDBGV4&client_secret=QBLN55MW0KA5WSAMHGUSRMDCNGWBGT2I3XL3ANP0BP4NUIG0&v=20180605&ll=43.80225000000007,-79.35557999999997&radius=500&limit=100'

In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60015386e93b3e0bd504339f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.806750004500074,
    'lng': -79.34935664290906},
   'sw': {'lat': 43.79774999550007, 'lng': -79.36180335709089}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f2e83dbe4b062ad2c20dd96',
       'name': 'Woodbrooke Estate',
       'location': {'address': '3740 Don Mills Road',
        'crossStreet': '& the Byways',
        'lat': 43.802067347853196,
        'lng': -79.35434683847974,
        'l

In [51]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [54]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-54-bc3544d26c92>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Woodbrooke Estate,Residential Building (Apartment / Condo),43.802067,-79.354347
1,McNicoll Park,Park,43.798994,-79.352840


In [55]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


##  Explore Neighborhoods in North,York


In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:
NY_venues = getNearbyVenues(names=NY_data['Neighbourhood'],
                                   latitudes=NY_data['Latitude'],
                                   longitudes=NY_data['Longitude']
                                  )


Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [59]:
print(NY_venues.shape)
NY_venues.head()

(212, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.80225,-79.35558,Woodbrooke Estate,43.802067,-79.354347,Residential Building (Apartment / Condo)
1,Hillcrest Village,43.80225,-79.35558,McNicoll Park,43.798994,-79.352840,Park
2,"Fairview, Henry Farm, Oriole",43.69208,-79.47863,McDonald's,43.692926,-79.479982,Fast Food Restaurant
3,"Fairview, Henry Farm, Oriole",43.69208,-79.47863,Subway,43.690218,-79.474050,Sandwich Place
4,"Fairview, Henry Farm, Oriole",43.69208,-79.47863,Dollar Tree,43.690296,-79.474667,Discount Store


In [60]:
NY_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Bayview Village,6,6,6,6,6,6
"Bedford Park, Lawrence Manor East",11,11,11,11,11,11
Don Mills,17,17,17,17,17,17
Downsview,37,37,37,37,37,37
"Fairview, Henry Farm, Oriole",6,6,6,6,6,6
Glencairn,16,16,16,16,16,16
Hillcrest Village,2,2,2,2,2,2
Humber Summit,2,2,2,2,2,2


In [61]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 51 uniques categories.


<h5>One Hot Encoding of Features</h5>

In [62]:
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

NY_onehot['Neighbourhood'] = NY_venues['Neighbourhood'] 

fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Baby Store,Bakery,Bank,Bar,Baseball Field,Beer Store,...,Pizza Place,Pub,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Shopping Mall,Sporting Goods Shop,Supermarket,Sushi Restaurant,Vietnamese Restaurant
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
NY_onehot.shape

(212, 52)

#### Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category


In [64]:
NY_grouped = NY_onehot.groupby('Neighbourhood').mean().reset_index()
NY_grouped


,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Baby Store,Bakery,Bank,Bar,Baseball Field,Beer Store,...,Pizza Place,Pub,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Shopping Mall,Sporting Goods Shop,Supermarket,Sushi Restaurant,Vietnamese Restaurant
0,"Bathurst Manor, Wilson Heights, Downsview North",0.00,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000
1,Bayview Village,0.00,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.0000,0.0,0.166667,0.166667,0.000000,0.0,0.000000,0.00,0.000000
2,"Bedford Park, Lawrence Manor East",0.00,0.0000,0.00,0.000000,0.0000,0.181818,0.00,0.090909,0.000000,...,0.000000,0.0000,0.0,0.000000,0.090909,0.090909,0.0,0.090909,0.00,0.000000
3,Don Mills,0.00,0.0000,0.00,0.000000,0.0000,0.117647,0.00,0.058824,0.000000,...,0.000000,0.0000,0.0,0.058824,0.117647,0.058824,0.0,0.058824,0.00,0.000000
4,Downsview,0.00,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.027027,...,0.054054,0.0000,0.0,0.081081,0.108108,0.054054,0.0,0.000000,0.00,0.027027
5,"Fairview, Henry Farm, Oriole",0.00,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.0000,0.0,0.166667,0.166667,0.000000,0.0,0.000000,0.00,0.000000
6,Glencairn,0.00,0.0625,0.00,0.000000,0.0625,0.062500,0.00,0.000000,0.000000,...,0.187500,0.0625,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000
7,Hillcrest Village,0.00,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.0000,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000
8,Humber Summit,0.00,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.5,0.000000,0.00,0.000000
9,"Humberlea, Emery",0.00,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.0000,0.0,0.166667,0.166667,0.000000,0.0,0.000000,0.00,0.000000


<h5> Let's print each neighborhood along with the top 5 most common venues</h5>


In [65]:
num_top_venues = 5

for hood in NY_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0         Home Service   1.0
1  American Restaurant   0.0
2          Music Store   0.0
3          IT Services   0.0
4       Ice Cream Shop   0.0


----Bayview Village----
                  venue  freq
0  Fast Food Restaurant  0.33
1   Fried Chicken Joint  0.17
2        Sandwich Place  0.17
3            Restaurant  0.17
4        Discount Store  0.17


----Bedford Park, Lawrence Manor East----
            venue  freq
0     Coffee Shop  0.18
1            Bank  0.18
2  Ice Cream Shop  0.09
3  Baseball Field  0.09
4  Cosmetics Shop  0.09


----Don Mills----
                  venue  freq
0           Coffee Shop  0.12
1  Fast Food Restaurant  0.12
2                  Bank  0.12
3        Sandwich Place  0.12
4          Burger Joint  0.06


----Downsview----
                  venue  freq
0  Fast Food Restaurant  0.19
1        Discount Store  0.14
2        Sandwich Place  0.11
3   Fried Chicken Joint  0.11
4     

First, let's write a function to sort the venues in descending order.


In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = NY_grouped['Neighbourhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Home Service,Vietnamese Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Falafel Restaurant,Drugstore,Discount Store,Dessert Shop,Cosmetics Shop
1,Bayview Village,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
2,"Bedford Park, Lawrence Manor East",Bank,Coffee Shop,Burger Joint,Supermarket,Shopping Mall,Sandwich Place,Ice Cream Shop,Baseball Field,Cosmetics Shop,Cafeteria
3,Don Mills,Fast Food Restaurant,Bank,Sandwich Place,Coffee Shop,Discount Store,Cosmetics Shop,Ice Cream Shop,Fried Chicken Joint,Baseball Field,Burger Joint
4,Downsview,Fast Food Restaurant,Discount Store,Sandwich Place,Fried Chicken Joint,Restaurant,Shopping Mall,Pizza Place,Grocery Store,Vietnamese Restaurant,Liquor Store


<h3>K-Means Clustering Approach</h3>

Run _k_-means to cluster the neighborhood into 3 clusters.


In [74]:
kclusters = 3

NY_grouped_clustering = NY_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

kmeans.labels_

array([2, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [78]:
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

NY_merged = NY_data

NY_merged = NY_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

NY_merged.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.80225,-79.355580,0,0,0,Residential Building (Apartment / Condo),Park,Vietnamese Restaurant,Butcher,Fast Food Restaurant,Falafel Restaurant,Drugstore,Discount Store,Dessert Shop,Cosmetics Shop
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.69208,-79.478630,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
2,M2K,North York,Bayview Village,43.69208,-79.478630,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
3,M2L,North York,"York Mills, Silver Hills",43.74532,-79.387317,0,0,0,Bank,Coffee Shop,Burger Joint,Supermarket,Shopping Mall,Sandwich Place,Ice Cream Shop,Baseball Field,Cosmetics Shop,Cafeteria
4,M2M,North York,"Willowdale, Newtonbrook",43.79182,-79.413720,0,0,0,Korean Restaurant,Middle Eastern Restaurant,Café,Pizza Place,Dessert Shop,Hookah Bar,Sushi Restaurant,Park,Grocery Store,Bakery


Finally, let's visualize the resulting clusters


In [79]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighbourhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h5> Examine Clusters</h5>


In [97]:
NY_merged.loc[NY_merged['Cluster Labels'] == 0, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,0,0,Residential Building (Apartment / Condo),Park,Vietnamese Restaurant,Butcher,Fast Food Restaurant,Falafel Restaurant,Drugstore,Discount Store,Dessert Shop,Cosmetics Shop
3,North York,0,0,0,Bank,Coffee Shop,Burger Joint,Supermarket,Shopping Mall,Sandwich Place,Ice Cream Shop,Baseball Field,Cosmetics Shop,Cafeteria
4,North York,0,0,0,Korean Restaurant,Middle Eastern Restaurant,Café,Pizza Place,Dessert Shop,Hookah Bar,Sushi Restaurant,Park,Grocery Store,Bakery
6,North York,0,0,0,Bank,Coffee Shop,Burger Joint,Supermarket,Shopping Mall,Sandwich Place,Ice Cream Shop,Baseball Field,Cosmetics Shop,Cafeteria
7,North York,0,0,0,Grocery Store,Baby Store,Pizza Place,Park,Coffee Shop,Butcher,Café,Fried Chicken Joint,Fast Food Restaurant,Falafel Restaurant
9,North York,0,0,0,Fast Food Restaurant,Bank,Sandwich Place,Coffee Shop,Discount Store,Cosmetics Shop,Ice Cream Shop,Fried Chicken Joint,Baseball Field,Burger Joint
10,North York,0,0,0,Fast Food Restaurant,Bank,Sandwich Place,Coffee Shop,Discount Store,Cosmetics Shop,Ice Cream Shop,Fried Chicken Joint,Baseball Field,Burger Joint
12,North York,0,0,0,Coffee Shop,Restaurant,Pub,American Restaurant,Mexican Restaurant,Drugstore,IT Services,Café,Italian Restaurant,Japanese Restaurant
18,North York,0,0,0,Bank,Coffee Shop,Burger Joint,Supermarket,Shopping Mall,Sandwich Place,Ice Cream Shop,Baseball Field,Cosmetics Shop,Cafeteria
20,North York,0,0,0,Pizza Place,Grocery Store,Italian Restaurant,Mediterranean Restaurant,Pub,Gas Station,Japanese Restaurant,Latin American Restaurant,Fast Food Restaurant,Bakery


In [98]:
NY_merged.loc[NY_merged['Cluster Labels'] == 1, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
2,North York,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
5,North York,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
8,North York,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
13,North York,1,1,1,Fast Food Restaurant,Discount Store,Sandwich Place,Fried Chicken Joint,Restaurant,Shopping Mall,Pizza Place,Grocery Store,Vietnamese Restaurant,Liquor Store
14,North York,1,1,1,Fast Food Restaurant,Discount Store,Sandwich Place,Fried Chicken Joint,Restaurant,Shopping Mall,Pizza Place,Grocery Store,Vietnamese Restaurant,Liquor Store
15,North York,1,1,1,Fast Food Restaurant,Discount Store,Sandwich Place,Fried Chicken Joint,Restaurant,Shopping Mall,Pizza Place,Grocery Store,Vietnamese Restaurant,Liquor Store
16,North York,1,1,1,Fast Food Restaurant,Discount Store,Sandwich Place,Fried Chicken Joint,Restaurant,Shopping Mall,Pizza Place,Grocery Store,Vietnamese Restaurant,Liquor Store
17,North York,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop
19,North York,1,1,1,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Discount Store,Butcher,Falafel Restaurant,Drugstore,Dessert Shop,Cosmetics Shop


In [99]:
NY_merged.loc[NY_merged['Cluster Labels'] == 2, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,North York,2,2,2,Home Service,Vietnamese Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Falafel Restaurant,Drugstore,Discount Store,Dessert Shop,Cosmetics Shop


<p>Conclusion: 
   
Purpose of the project was to identify best neighborhood in North York,Toronto.I narrowed down the search for optimal location for hotel,school,malls etc.I used Foursquare data for searching nearby venues.Using K-means clustring algorithm I separated the neighborhood into  different clusters and top 10 venues for each neighborhood.Used geocoder to get the latitude and longitude coordinates of a given postal code.Got 103 different lattitude and logitude for the dataset.</p>
    
    
    
    
    
    
    
    
    
